In [101]:
import requests
import json 
import os
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd
from pandas import json_normalize
import datetime

from pyspark.sql import SparkSession
#from pyspark.sql.types import StructType



In [120]:
# ===================================== DESAFIO 2 =======================================================#

today = datetime.date.today()

res = requests.get('https://api.mercadolibre.com/sites/MLA/search?category=MLA1000')
response = json.loads(res.text)

NombreApi = 'MLA'
Formato = 'json'
AnoMes= today.strftime("%Y%m")

folder = NombreApi+Formato+AnoMes
dirName = os.path.dirname(os.path.realpath('__file__'))+'/'+folder


if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

    
with open(dirName+'/'+'response.json', 'w', encoding='utf-8') as f:
    json.dump(response, f, ensure_ascii=False, indent=4)

Directory  /home/cdsw/Challenge/MLAjson202207  already exists


In [110]:
# ===================================== DESAFIO 3 =======================================================#

gdd.download_file_from_google_drive(file_id='1mZV6T5p_YUnIPlkQzK1q9-rNJpgV6AIA',
                                    dest_path='./data/Sellers.json')

spark = SparkSession.builder.master('local').appName('Pablo').getOrCreate()
sc = spark.sparkContext

with open('./data/Sellers.json') as data_file:    
    data = json.load(data_file)  

df_json = json_normalize(data)

df_dataset = df_json[['body.site_id','body.id','body.nickname','body.points']].copy()

df_dataset.rename(columns= {'body.site_id':'siteId',
                            'body.id':'sellerId',
                            'body.nickname':'sellerNickname',
                            'body.points':'sellerPoints'}, inplace=True)

sdf_dataset = spark.createDataFrame(df_dataset)

sdf.show()

def generate_file(outdir,outname, df):

    if not os.path.exists(outdir):
        os.makedirs(outdir)

    fullname = os.path.join(outdir, outname)    

    df.write.csv(fullname)


today = datetime.date.today()
outname = 'sellerPoints.csv'

outdir= './MPE/'+today.strftime('%Y/%m/%d')+'/positivo'
generate_file(outdir,outname, sdf_dataset.filter(sdf_dataset['sellerPoints'] > 0))


outdir= './MPE/'+today.strftime('%Y/%m/%d')+'/cero'
generate_file(outdir,outname, sdf_dataset.filter(sdf_dataset['sellerPoints'] == 0))


outdir= './MPE/'+today.strftime('%Y/%m/%d')+'/negativo'
generate_file(outdir,outname, sdf_dataset.filter(sdf_dataset['sellerPoints'] < 0))



+------+---------+--------------------+------------+
|siteId| sellerId|      sellerNickname|sellerPoints|
+------+---------+--------------------+------------+
|   MPE|298734964|      MARIELATAQUIRE|           2|
|   MPE|183049329|         MURO8709951|          -3|
|   MPE| 94592189|          ILLARYPERU|          -2|
|   MPE|520133997|     ISABELLADELPOZO|           1|
|   MPE|684964436|         PHMO1747353|           0|
|   MPE|685079498|MELISSASUSANAARVA...|           0|
|   MPE|646068761|   YOMIDELGADOSNCHEZ|           0|
|   MPE|685310649|DONATILDONATILDEC...|           0|
|   MPE|685419864|        VANESSAURNER|           0|
|   MPE|285674870|    ERICKLOPEZUSMSYA|           2|
|   MPE|685275449|DANIELARUIZRIDRIGUES|           0|
|   MPE| 48893023|        MARCELASUSAN|           1|
|   MPE|603331827|         COVA1031117|           0|
|   MPE|205264135|         GOBR7283790|           1|
|   MPE|580279940|        LORDVENCEDOR|           0|
|   MPE|300834652| DANIELAHILARIORAMOS|       

In [121]:
# ===================================== DESAFIO 4 =======================================================#
gdd.download_file_from_google_drive(file_id='1t_DTfRKdeJv1djN6R79Nxwb5yzE3pMx1',
                                    dest_path='./data/MPE1004.json')

with open('./data/MPE1004.json') as data_file:    
    data = json.load(data_file)  

df_json = json_normalize(data).explode('results')
df_json_2 = json_normalize(df_json['results'])

df_json_2['rowId'] = df_json_2.index

df_dataset_4 = df_json_2[['rowId','id','sold_quantity','available_quantity']].copy()

df_dataset_4.rename(columns= {'id':'itemId',
                            'sold_quantity':'soldQuantity',
                            'available_quantity':'availableQuantity'}, inplace=True)

df_dataset_4

,rowId,itemId,soldQuantity,availableQuantity
0,0,MPE433108265,6,9
1,1,MPE434382765,6,3
2,2,MPE433853177,3,17
3,3,MPE419883282,15,18
4,4,MPE431714651,15,1
5,5,MPE438492919,0,100
6,6,MPE429448587,0,50
7,7,MPE439307195,0,3
8,8,MPE439307251,0,3
9,9,MPE437503507,0,10


In [122]:
# ===================================== DESAFIO 5 =======================================================#

gdd.download_file_from_google_drive(file_id='1iMjqXR30iX6NRdhVEBuv_7ciUbmLz4eO',
                                    dest_path='./data/visits.csv')

df_visits = pd.read_csv('./data/visits.csv')

df_dataset_5 = df_dataset_4[['itemId','soldQuantity']].copy()

df_merge = pd.merge(df_dataset_5, df_visits, on = 'itemId', how='inner')

df_merge = df_merge.loc[df_merge['soldQuantity']!=0]

df_merge

,itemId,soldQuantity,visits
0,MPE433108265,6,203
1,MPE434382765,6,170
2,MPE433853177,3,1034
3,MPE419883282,15,1772
4,MPE431714651,15,33
14,MPE431446248,2,2242
21,MPE432990777,1,426
22,MPE440389411,1,158
23,MPE421767433,4,746
25,MPE432439269,2,42


In [123]:
# ===================================== DESAFIO 6 =======================================================#

df_merge['conversionRate'] = df_merge['soldQuantity']/df_merge['visits']

df_merge['conversionRanking'] = df_merge['conversionRate'].rank(ascending = 1)

df_merge.sort_values(by = 'conversionRate', inplace = True)

df_merge.reset_index(inplace = True, drop = True)

df_merge

,itemId,soldQuantity,visits,conversionRate,conversionRanking
0,MPE431446248,2,2242,0.000892,1.0
1,MPE432990777,1,426,0.002347,2.0
2,MPE428549082,1,352,0.002841,3.0
3,MPE433853177,3,1034,0.002901,4.0
4,MPE421767433,4,746,0.005362,5.0
5,MPE440389411,1,158,0.006329,6.0
6,MPE419883282,15,1772,0.008465,7.0
7,MPE432728801,1,68,0.014706,8.0
8,MPE430002527,1,60,0.016667,9.0
9,MPE433933924,1,49,0.020408,10.0


In [124]:
# ===================================== DESAFIO 7 =======================================================#

df_dataset_7 = df_dataset_4[['itemId','availableQuantity']].copy()

df_dataset_7['stockPercentage'] = (df_dataset_7['availableQuantity']/df_dataset_7['availableQuantity'].sum())*100

df_dataset_7.sort_values(by = 'stockPercentage', ascending= False, inplace= True)

df_dataset_7.reset_index(inplace = True, drop = True)

df_dataset_7

,itemId,availableQuantity,stockPercentage
0,MPE433046443,999,70.302604
1,MPE436649728,100,7.037298
2,MPE438492919,100,7.037298
3,MPE429448587,50,3.518649
4,MPE431446248,23,1.618578
5,MPE419883282,18,1.266714
6,MPE433853177,17,1.196341
7,MPE421767433,11,0.774103
8,MPE432202936,10,0.703730
9,MPE437503507,10,0.703730


In [131]:
# ===================================== DESAFIO 8 =======================================================#

def generateMonthlyPathList(year, month, day):
    end= int(day)
    start= 1

    date_list = []

    for x in range(start,end+1):
        path = 'https://importantdata@location/'+year+'/'+month+'/'+str(x)+'/'
        date_list.append(path)
 
    return date_list

year= input("Ingrese el año 'yyyy': \n")
month= input("Ingrese el mes 'mm': \n")
day= input("Ingrese el dia 'dd': \n")

path_list = []

path_list = generateMonthlyPathList(year, month, day)
    
for x in path_list:
    print(x)

Ingrese el año 'yyyy': 
2022
Ingrese el mes 'mm': 
01
Ingrese el dia 'dd': 
12
https://importantdata@location/2022/01/1/
https://importantdata@location/2022/01/2/
https://importantdata@location/2022/01/3/
https://importantdata@location/2022/01/4/
https://importantdata@location/2022/01/5/
https://importantdata@location/2022/01/6/
https://importantdata@location/2022/01/7/
https://importantdata@location/2022/01/8/
https://importantdata@location/2022/01/9/
https://importantdata@location/2022/01/10/
https://importantdata@location/2022/01/11/
https://importantdata@location/2022/01/12/


In [126]:
# ===================================== DESAFIO 9 =======================================================#
from datetime import datetime, timedelta

def generateLastDaysPaths(date_str, days):
   
    date_list = []

    start_date = datetime.strptime(date_str, '%Y%m%d')

    for x in range(0,days):
        date = start_date - timedelta(days=x)
        year= date.strftime('%Y')
        month= date.strftime('%m')
        day = date.strftime('%d')

        path = 'https://importantdata@location/'+year+'/'+month+'/'+day+'/'
        date_list.append(path)

    return date_list

year= input("Ingrese la fecha inicial 'yyyymmdd': \n")
days= int(input("Ingrese los dias 'dd': \n"))

date_list = []

date_list = generateLastDaysPaths(date, days)
    
for x in date_list:
    print(x)

Ingrese la fecha inicial 'yyyymmdd': 
202201017
Ingrese los dias 'dd': 
2
https://importantdata@location/2021/04/11/
https://importantdata@location/2021/04/10/
